In [1]:
import nbimporter
import numpy as np
import MoistureConversion as mc
import vapour_pressure as vp
import BentheimerSandstone

Importing Jupyter notebook from MoistureConversion.ipynb
Importing Jupyter notebook from BentheimerSandstone.ipynb
Importing Jupyter notebook from vanGenuchten.ipynb


### 物性値を取得するクラス  
各物性値ファイルに存在する物性値を元の形式のまま取得してくるクラス。 

### 取得した物性値を流量計算に適した形に換算するクラス

#### 実験から熱水分物性値として考えられるもの  
【参考】建築材料の熱・空気・湿気物性値，日本建築学会，丸善株式会社，2001など

#### 熱物性値  
$\dot q$：単位時間当たりの熱流量の密度[W/m2]  
$\rho_0$：密度[kg/m3]  
$c_0$：比熱[J/kg K]  
$c_0\rho_0$：容積熱容量[J/m3 K]  

$\lambda$：熱伝導率[W/mK]  
定義：$\dot q = -\lambda・grad\space T$  

$R$：熱抵抗[Km2/W]  

#### 水分物性値  
$\dot m_v$：単位時間当たりの水蒸気流量の密度[kg/m2 s]  
$\dot m_m$：単位時間当たりの水分流量の密度[kg/m2 s]  
$\phi$：含水率[m3/m3]  
他標記として$w$[kg/m3]、$u$[kg/kg]など  
$S$：飽和度[-]  
#### 気相水分伝導率  
$\delta_p$：透湿率[kg/m Pa s]　文中記号DP  
定義：$m_v = -\delta_p・grad\space p_v$  

$Z_p$：湿気抵抗[m2Pa s/kg]  
$\delta_v$：水蒸気の拡散係数[m2/s]  
定義：$\dot m_v = -\delta_v・grad\space \rho_v$

$k_T$：温度勾配水分伝導率[kg/m K s]  
定義：$\dot m_m = -k_T・grad\space T$  

$D_T$：温度勾配水分拡散係数[m2/K s]  
定義：$\dot m_m = -\rho_0 D_T・grad\space T$  

$\lambda^{'}_{\mu g}$：水分化学ポテンシャル勾配に対する気相水分伝導率[kg/ms (J/kg)]  
定義：$\dot m_v = -\lambda^{'}_{\mu g}・grad\space \mu$

$\lambda^{'}_{Tg}$：温度勾配に対する気相水分伝導率[kg/ms K]  
定義：$\dot m_v = -\lambda^{'}_{Tg}・grad\space \mu$

#### 液相水分伝導率  
$k_m$：水分伝導率[kg/m Pa s]  
定義：$\dot m_m = -k_m・grad\space s$

$D_w$：水分拡散係数[m2/s]  
定義：$\dot m_m = -\rho_0 D_w・grad\space u$  

$\lambda^{'}_{\mu l}$：水分化学ポテンシャル勾配に対する液相水分伝導率[kg/ms (J/kg)]  
定義：$\dot m_v = -\lambda^{'}_{\mu l}・grad\space \mu$

#### 空気の拡散係数  
$k_a$：透気率[m2/Ks]  


※新しい基準で物性値を測定した場合、本クラスに書き加えること


In [2]:
def getCRow():
    return self.material()

In [3]:
class PhysicalProperty(): 
    
    Rv = 8316.0/18.016
    roww = 1000.0 # kg/m3
    grav = 9.806650
    roww = 1000.0 
    r    = 4.18605E+3
    
    def __init__( self, material, temp, moisture ):
        
        self.temp = temp
        self.moisture = moisture
        
        if moisture == 0.0:
            # 水分の計算をしない場合
            self.moisture = 'NaN'
        elif moisture < 0.0:
            # 水分化学ポテンシャル
            self.miu = moisture 
            self.rh  = mc.cal_MiutoRH( temp, moisture )
            self.pv  = mc.cal_MiutoPv( temp, moisture )
        elif 0.0 < moisture and moisture <= 1.0:
            # 相対湿度
            self.rh  = moisture 
            self.miu = mc.cal_RHtoMiu( temp, moisture )
            self.pv  = mc.cal_RHtoPv(  temp, moisture )
        elif 1.0 < moisture:
            # 水蒸気圧
            self.pv  = moisture 
            self.miu = mc.cal_PvtoMiu( temp, moisture )
            self.rh  = mc.cal_PvtoRH(  temp, moisture )
        else : 
            raise ValueErrow('水分の値が想定の範囲外となってます')
        
        # 新しく物性モジュールを作成した場合は、ここに追加してください。
        mlist = {
        #        '合板' : 'Plywood', #物性値ファイルの名前
        #        'グラスウール' : 'GlassWool',
                'BentheimerSandstone': BentheimerSandstone.Property( temp, self.miu )
        }
        
        self.material = mlist[material]
        self.list = self.material.proplist
    
##########################################################
#   熱容量
    def CRow( self ):
        if 'crow' in self.list:  
            if self.moisture == 'NaN':
                return self.material.getCRow() # 含水率による増分はここに入れる
            else :
                return self.material.getCRow() + self.roww * self.material.getPhi() * self.r   
        
#   熱伝導率
    def LAM( self ):
        if 'lam' in self.list:
            return self.material.getLAM()
    
##########################################################
#   水分容量（気相の水分状態（ポテンシャル・相対湿度など）から含水率への変換）
    def Phi( self ):
        if 'phi' in self.list:
            return self.material.getPhi() 
        
#   含水率から水分化学ポテンシャルへの変換　#相対湿度・水蒸気圧などへの換算も増やそう
    def Miu( self ):
        if 'miu' in self.list:
            return self.material.getMiu()
        
#   含水率の水分化学ポテンシャル微分
    def DPhi( self ):
        if 'dphi' in self.list:
            return self.material.getDPhi()
        
###     気相水分伝導率    #######
#   水蒸気圧勾配に対する気相水分伝導率（透湿率）
    def DP( self ):
        if 'dp' in self.list:
            return self.material.getDP()

#   水分化学ポテンシャル勾配に対する気相水分伝導率
    def LDMG( self ):
        if 'ldmg' in self.list:
            return self.material.getLDMG()
        elif 'dp' in self.list:
            DPDM = vp.Pvs( tem ) * np.exp( self.miu / self.Rv/ self.tem ) / self.Rv / self.tem
            return DPDM * self.material.getDP()
        
#   温度勾配に対する気相水分伝導率
    def LDTG( self ):
        if 'ldtg' in self.list:
            return self.material.getLDTG()
        elif 'dp' in self.list:    
            DPDT = np.exp( self.Miu / self.Rv / self.tem ) * ( vp.DPvs( self.tem ) - vp.Pvs( self.tem ) * self.Miu / self.Rv / ( self.tem**2 ))
            return DPDT * self.material.getDP()
    
###     液相水分伝導率    #######
#   水分化学ポテンシャル勾配に対する液相水分伝導率
    def LDML( self ):
        if 'ldml' in self.list:
            return self.material.getLDML()
        elif 'dw' in self.list:
            return self.roww / self.grav * self.material.getDw() 

################################################      


In [7]:
# 使用例
temp = 284.15
miu  = -10000.0
stone = PhysicalProperty( 'BentheimerSandstone', temp, miu )
stone.material.proplist
print(stone.list)
print(stone.LDML())
#print(stone.list)

['crow', 'lam', 'phi', 'miu', 'dw', 'dp', 'dphi']
1.472197037026832e-19


In [ ]:
p1 = PropertyConversion('BentheimerSandstone')#,'RH')
p2 = PropertyConversion('BentheimerSandstone','Miu')
p1.LDMLFromRH(300.0,0.9)
p2.LDML(300.0,-100.0)